In [1]:
# Workflow and process 

In [2]:
# Steps
# 1. import the csv, clean it up, geocode it and get the coordinates 
# 2. use mapbox to build the maps and adobe
#     2a. choose a colour pallete and font family stuff 
#     - canada wide
#     - east coast highlight 
#     - halifax highlight (adobe) 
#     - common highlight (adobe?)
#     - facebook marketplace highlight (adobe) 
#     - survey respondents - where they live, how they call it
#     - flower streets and dartmouth 
#     - north end 
# 3. build out in datawrapper an interactive click of people's responses and then what they said for their neighbourhood 
#     - can this show frequency or closeness at all?
# 4. layer from halifax geo map or something????
# 5. build out html scrolly 
# 6. learn how to publish 

In [3]:
import pandas as pd
import numpy as np
import warnings, os
import re

In [4]:
#import the data in order to clean it up and geocode it 

In [5]:
data = pd.read_csv('data-jun-27.csv')

In [6]:
data.head()

,Timestamp,What is your address?,How do you call or refer to the neighbourhood or area where you live?,Is there a secondary way you refer to where you live? Share here,Do you have further thoughts about how you refer to where you live?,Are you interesting in telling me more about this topic?,"If yes, please provide your name and an email address I can reach you at."
0,21/06/2025 09:28:19,2351 Maynard Street,The north end,Halifax,I have never said HRM in my life,Yes,tara.thorne@gmail.com
1,21/06/2025 09:29:12,Ford St,Fairview,The Peninsula's armpit,No,Yes,Calebschriver@gmail.com
2,21/06/2025 09:29:58,76 main ave,Fairview,Halifax,I don't but I'm happy to talk to you about it,Yes,Kdwakeman@gmail.com
3,21/06/2025 09:44:53,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,Apparently it was known historically as Austen...,Yes,"Jacob Boon, jboon@thetyee"
4,21/06/2025 09:46:08,36 Hansens,waverley,my road,I'm in a secluded lil area w maybe not a Lot o...,NaN,NaN


In [7]:
data.columns


Index(['Timestamp', 'What is your address? ',
       'How do you call or refer to the neighbourhood or area where you live?',
       'Is there a secondary way you refer to where you live? Share here',
       'Do you have further thoughts about how you refer to where you live?',
       'Are you interesting in telling me more about this topic?',
       'If yes, please provide your name and an email address I can reach you at. '],
      dtype='object')

In [8]:
#rename the columns that are being kept
data_cleaned = data.rename(columns={"What is your address? ":"Address", "How do you call or refer to the neighbourhood or area where you live?":"Name", "Is there a secondary way you refer to where you live? Share here":"Secondary name" })

In [9]:
#drop columns I dont need 
data_cleaned.drop(columns= ['Timestamp','Do you have further thoughts about how you refer to where you live?',
       'Are you interesting in telling me more about this topic?',
       'If yes, please provide your name and an email address I can reach you at. '], inplace=True)

In [10]:
data_cleaned.head()

,Address,Name,Secondary name
0,2351 Maynard Street,The north end,Halifax
1,Ford St,Fairview,The Peninsula's armpit
2,76 main ave,Fairview,Halifax
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth
4,36 Hansens,waverley,my road


In [11]:
data_cleaned["Name"].nunique()

24

In [12]:
data_cleaned["Name"].unique()

array(['The north end', 'Fairview', 'Flower Streets', 'waverley',
       'Downtown ', 'North end ', 'north end ',
       'Dartmouth, Portland Street area, like toward Cole harbour way. ',
       'Downtown Dartmouth', 'Larry Uteck', 'Bedford ', 'West end',
       'North End', 'Downtown Dartmouth ', 'Sullivan’s Pond',
       'The North End ', 'Quinpool', 'Bedford',
       'I usually say "Central Halifax" ', 'the South End',
       'West bedford', 'The West End', 'Larry Utech',
       '(Bottom of the) North End'], dtype=object)

In [13]:
data_cleaned[data_cleaned['Name'].str.contains("North end", na=False)]

,Address,Name,Secondary name
6,2494 John Street,North end,“Near Agricola” or “near the commons”
29,6143 Charles Street,North end,near the common


In [14]:
#get rid of spaces, get rid of caps 
data_cleaned['Area'] = data_cleaned['Name'].str.lower().str.strip()

In [15]:
data_cleaned

,Address,Name,Secondary name,Area
0,2351 Maynard Street,The north end,Halifax,the north end
1,Ford St,Fairview,The Peninsula's armpit,fairview
2,76 main ave,Fairview,Halifax,fairview
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,flower streets
4,36 Hansens,waverley,my road,waverley
5,1583 Hollis St,Downtown,NaN,downtown
6,2494 John Street,North end,“Near Agricola” or “near the commons”,north end
7,2494 John St,north end,off agricola,north end
8,"18 Hamlet Lane, Dartmouth, NS.","Dartmouth, Portland Street area, like toward C...","Dartmouth, 5 minutes to Cole Harbour.","dartmouth, portland street area, like toward c..."
9,70 Ochterloney Street,Downtown Dartmouth,NaN,downtown dartmouth


In [16]:
#creating a function to sort the answers into larger, standardized buckets (there must be a simpler way, but I like that I can understand what's happening here) 
def generalize_area(name):
    if 'north end' in name:
        return 'north end'
    elif 'south end' in name:
        return 'south end'
    elif 'downtown dartmouth' in name:
        return 'downtown dartmouth'
    elif 'downtown' in name:
        return 'downtown'
    elif 'west end' in name:
        return 'west end'
    elif 'larry uteck' in name or 'larry utech' in name:
        return 'larry uteck'
    elif 'bedford' in name:
        return 'bedford'
    elif 'fairview' in name:
        return 'fairview'
    elif 'waverley' in name:
        return 'waverley'
    elif 'quinpool' in name:
        return 'west end' #this one was manual
    elif 'central halifax' in name:
        return 'central halifax'
    elif 'sullivan' in name and 'pond' in name:
        return "dartmouth" #this one was manual
    elif 'flower streets' in name:
        return 'dartmouth' #this one was manual
    elif 'dartmouth' in name:
        return 'dartmouth'
    else:
        return 'unknown'


In [17]:
#applying the above function to the Name_cleaned column, giving the bigger buckets
data_cleaned['Area'] = data_cleaned['Area'].apply(generalize_area)

In [18]:
data_cleaned.head()

,Address,Name,Secondary name,Area
0,2351 Maynard Street,The north end,Halifax,north end
1,Ford St,Fairview,The Peninsula's armpit,fairview
2,76 main ave,Fairview,Halifax,fairview
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,dartmouth
4,36 Hansens,waverley,my road,waverley


In [19]:
data_cleaned['Area'].value_counts()

Area
north end             7
fairview              4
downtown dartmouth    4
dartmouth             3
bedford               3
west end              3
larry uteck           2
waverley              1
downtown              1
central halifax       1
south end             1
Name: count, dtype: int64

In [20]:
#I'm satisfied with those buckets for further storytelling, now I'll clean the addresses so I can get the geocoordinates for everyone's addresses if I can

In [21]:
address_parts = data_cleaned['Address'].str.split(',', expand=True)

In [22]:
data_cleaned['Street'] = address_parts[0].str.strip()
data_cleaned['City'] = address_parts[1].str.strip() if 1 in address_parts.columns else ''
data_cleaned['Province'] = address_parts[2].str.strip() if 2 in address_parts.columns else ''

In [23]:
data_cleaned['City'] = data_cleaned['City'].replace('', pd.NA)
data_cleaned['City'] = data_cleaned['City'].fillna('Halifax')
data_cleaned['Province'] = 'NS'
data_cleaned['Country'] = 'Canada'

In [24]:
data_cleaned['Address_cleaned'] = data_cleaned['Address'].astype(str).str.strip()

In [25]:
data_cleaned.head()

,Address,Name,Secondary name,Area,Street,City,Province,Country,Address_cleaned
0,2351 Maynard Street,The north end,Halifax,north end,2351 Maynard Street,Halifax,NS,Canada,2351 Maynard Street
1,Ford St,Fairview,The Peninsula's armpit,fairview,Ford St,Halifax,NS,Canada,Ford St
2,76 main ave,Fairview,Halifax,fairview,76 main ave,Halifax,NS,Canada,76 main ave
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,dartmouth,53 Tulip Street,Dartmouth,NS,Canada,"53 Tulip Street, Dartmouth"
4,36 Hansens,waverley,my road,waverley,36 Hansens,Halifax,NS,Canada,36 Hansens


In [28]:
# Combine into a single full address column
data_cleaned['Full_address'] = data_cleaned['Street'] + ', ' + data_cleaned['City'] + ', ' + data_cleaned['Province'] + ', ' + data_cleaned['Country']


In [29]:
# Convert to list for geocoding
address_list = data_cleaned['Full_address'].tolist()

In [30]:
street_list = data_cleaned['Street'].tolist()

In [31]:
city_list = data_cleaned['City'].tolist()

In [32]:
len(address_list)

30

In [33]:
print(data_cleaned['Full_address'])

0              2351 Maynard Street, Halifax, NS, Canada
1                          Ford St, Halifax, NS, Canada
2                      76 main ave, Halifax, NS, Canada
3                53 Tulip Street, Dartmouth, NS, Canada
4                       36 Hansens, Halifax, NS, Canada
5                   1583 Hollis St, Halifax, NS, Canada
6                 2494 John Street, Halifax, NS, Canada
7                     2494 John St, Halifax, NS, Canada
8                 18 Hamlet Lane, Dartmouth, NS, Canada
9            70 Ochterloney Street, Halifax, NS, Canada
10            635 Larry Uteck blvd, Halifax, NS, Canada
11            Ochterloney Street, Dartmouth, NS, Canada
12        23 bridge street Bedford, Halifax, NS, Canada
13              3208 Connaught Ave, Halifax, NS, Canada
14                   5274 Glebe St, Halifax, NS, Canada
15    33 Prince Albert Rd. Dartmouth, Halifax, NS, C...
16                33 Prince Albert, Halifax, NS, Canada
17               84 Crichton Ave, Dartmouth, NS,

In [42]:
#now I will see if these are clean enough to be geocoded
import requests
params = {
    'api_key': '8eeb81e1ae1860eb9e8a785155ef86e1b76171e',
    'q' : address_list
    # 'street': street_list,
    # 'city': city_list,
    # 'state': "NS",
    # 'country': 'Canada'
}
response = requests.get('https://api.geocod.io/v1.7/geocode', params=params)

if response.status_code != 200:
    raise Exception(f"Geocodio API error: {response.status_code} - {response.text}")

results = response.json()['results']


In [43]:
len(results)

5